In [1]:
#import
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import shap

C:\Users\noton\anaconda3\envs\TN5\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Reading dataset
df=pd.read_csv('Hybrid.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Cate_name          68 non-null     object 
 1   Cate_HOMO          68 non-null     float64
 2   Cate_LUMO          68 non-null     float64
 3   Cate_MC_O1         68 non-null     float64
 4   Cate_MC_H1         68 non-null     float64
 5   Cate_MC_O2         68 non-null     float64
 6   Cate_MC_H2         68 non-null     float64
 7   Cate_BL_ArO1       68 non-null     float64
 8   Cate_BL_O1H1       68 non-null     float64
 9   Cate_BL_ArO2       68 non-null     float64
 10  Cate_BL_O2H2       68 non-null     float64
 11  Cate_total_dipole  68 non-null     float64
 12  Cate_RDKit_PC_0    68 non-null     float64
 13  Cate_RDKit_PC_1    68 non-null     float64
 14  Cate_RDKit_PC_2    68 non-null     float64
 15  Cate_RDKit_PC_3    68 non-null     float64
 16  Cate_RDKit_PC_4    68 non-nu

In [3]:
#Building descriptors
X = df.drop(columns=['Yield', 'Cate_name', 'P_name'])
print('---Descriptors---')
print(X.head())

y = pd.DataFrame(df['Yield'],columns=['Yield'])
y=y.astype(int)
print('---Objective---')
print(y.head())

---Descriptors---
   Cate_HOMO  Cate_LUMO  Cate_MC_O1  Cate_MC_H1  Cate_MC_O2  Cate_MC_H2  \
0  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
1  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
2  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
3  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
4  -5.807916  -0.157824   -0.376813    0.263887   -0.418652    0.276114   

   Cate_BL_ArO1  Cate_BL_O1H1  Cate_BL_ArO2  Cate_BL_O2H2  ...    P_HOMO  \
0       1.36318       0.96622       1.37543       0.96253  ... -8.171191   
1       1.36318       0.96622       1.37543       0.96253  ... -8.457723   
2       1.36318       0.96622       1.37543       0.96253  ... -7.460440   
3       1.36318       0.96622       1.37543       0.96253  ... -8.082211   
4       1.36386       0.96625       1.37631       0.96239  ... -8.171191   

     P_LUMO    P_MC_P   P_MC_=O    P_MC_H  P_BL_P=O   P_BL_PH  P_total_dip

In [4]:
#storage 
data_r2_train =[]
data_RMSE_train = []
data_MAE_train = []
data_r2_test = []
data_RMSE_test = []
data_MAE_test = []
best_model_parametors = []
data_cv_score = []
cum_imp=pd.Series(index = X.columns)
cum_imp.fillna(0, inplace=True)

#RF
for i in range(10):
    seed=i
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
        
    param_grid = {"max_depth":[2,3,4,5],
                  'n_estimators':[50,100,500,1000,3000]}
    reg_rf = GridSearchCV(RandomForestRegressor(random_state=0), param_grid=param_grid, cv=5, n_jobs=16)
    reg_rf.fit(X_train,y_train['Yield'])
    reg_best = reg_rf.best_estimator_
    y_pred1 = reg_best.predict(X_train)
    y_pred2 = reg_best.predict(X_test)
            
    #train
    r2_train = metrics.r2_score(y_train, y_pred1)
    RMSE_train = metrics.root_mean_squared_error(y_train, y_pred1)
    MAE_train =  metrics.mean_absolute_error(y_train, y_pred1)
    #test
    r2_test = metrics.r2_score(y_test, y_pred2)
    RMSE_test = metrics.root_mean_squared_error(y_test, y_pred2)
    MAE_test = metrics.mean_absolute_error(y_test, y_pred2)

    parametors = reg_rf.best_params_
    best_model_parametors.append(parametors)

    data_r2_train.append(r2_train)
    data_RMSE_train.append(RMSE_train)
    data_MAE_train.append(MAE_train)
    data_r2_test.append(r2_test)
    data_RMSE_test.append(RMSE_test)
    data_MAE_test.append(MAE_test)
    data_cv_score.append(reg_rf.best_score_)
    
    #cumulative importances 
    importances = pd.Series(reg_best.feature_importances_, index = X.columns)
    cum_imp += importances
    
    print('----------------------')
    print('seed:', seed)
    print("Best Model Parameter:",reg_rf.best_params_)
    print("Best Model Score:",reg_rf.best_score_)
    print('R2_test:', r2_test)
print('R2_train_means:', sum(data_r2_train)/10)
print('CV_score_means:', sum(data_cv_score)/10)
print('R2_test_means:', sum(data_r2_test)/10)

----------------------
seed: 0
Best Model Parameter: {'max_depth': 2, 'n_estimators': 50}
Best Model Score: 0.8501144583319696
R2_test: 0.7081607613998477
----------------------
seed: 1
Best Model Parameter: {'max_depth': 3, 'n_estimators': 100}
Best Model Score: 0.7496605090486237
R2_test: 0.8830623464032926
----------------------
seed: 2
Best Model Parameter: {'max_depth': 3, 'n_estimators': 50}
Best Model Score: 0.8268038502876032
R2_test: 0.8530152678677587
----------------------
seed: 3
Best Model Parameter: {'max_depth': 3, 'n_estimators': 50}
Best Model Score: 0.7347964653275342
R2_test: 0.890966934681614
----------------------
seed: 4
Best Model Parameter: {'max_depth': 5, 'n_estimators': 50}
Best Model Score: 0.8440677614148535
R2_test: 0.8079197681552652
----------------------
seed: 5
Best Model Parameter: {'max_depth': 4, 'n_estimators': 100}
Best Model Score: 0.8165863053609259
R2_test: 0.7872932541008911
----------------------
seed: 6
Best Model Parameter: {'max_depth': 2,

In [5]:
data_r2_train_pd = pd.DataFrame(data= data_r2_train, columns=['r2_train'])
data_RMSE_train_pd = pd.DataFrame(data=data_RMSE_train, columns=['RMSE_train'])
data_MAE_train_pd = pd.DataFrame(data=data_MAE_train, columns=['MAE_train'])
data_r2_test_pd = pd.DataFrame(data=data_r2_test, columns=['r2_test'])
data_RMSE_test_pd = pd.DataFrame(data=data_RMSE_test, columns=['RMSE_test'])
data_MAE_test_pd = pd.DataFrame(data=data_MAE_test, columns=['MAE_test'])
data_cv_score_pd = pd.DataFrame(data=data_cv_score, columns=['cv_score'])
data_parametors = pd.DataFrame([best_model_parametors])

data_all = pd.concat([data_r2_train_pd, data_RMSE_train_pd, data_MAE_train_pd, data_r2_test_pd, 
                      data_RMSE_test_pd, data_MAE_test_pd, data_cv_score_pd],
                     axis=1, join='inner')

data_all.loc['mean'] = data_all.mean()
data_all.loc['std'] = data_all.std(ddof=0)
print(data_all)

data_all.to_csv('../../score/Hybrid/RF_score.csv')

      r2_train  RMSE_train  MAE_train   r2_test  RMSE_test   MAE_test  \
0     0.884168    9.241142   5.044802  0.708161  19.537430  12.718244   
1     0.919938    8.508852   4.918763  0.883062  10.644004   6.993067   
2     0.937808    7.585083   4.379750  0.853015  11.922384   7.349836   
3     0.942210    7.341781   4.221297  0.890967  10.229568   7.172330   
4     0.968175    5.237968   3.352614  0.807920  14.291381   8.337515   
5     0.969569    4.964016   3.019574  0.787293  16.083992   9.904638   
6     0.852689   10.995209   6.176883  0.715252  18.511692  11.451420   
7     0.960967    6.110244   3.660894  0.808820  13.164569   9.342012   
8     0.873196   10.564180   5.929801  0.724637  16.496169  10.319585   
9     0.901035   10.014763   6.362350  0.941256   6.433874   4.287364   
mean  0.920975    8.056324   4.706673  0.812038  13.731506   8.787601   
std   0.037685    1.963203   1.073399  0.072668   3.676658   2.231985   

      cv_score  
0     0.850114  
1     0.749661  